In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -q '/content/drive/My Drive/NLP/kaggle/imdb-dataset-sentiment-analysis-in-csv-format.zip'

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
tr_data=pd.read_csv('/content/Train.csv')

In [ ]:
tr_data.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [ ]:
tr_data.shape

(40000, 2)

In [ ]:
import nltk
nltk.download('punkt')
import string
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
class text_preprocessing:
  def __init__(self):
    pass
  # Convert text into lower case
  def convert_to_lower(self, text):
    return text.lower()
 
  # Remove punctuation from text
  def remove_punctuation(self, text):
 
    return "".join([c for c in text if c not in string.punctuation])
  # Tokenize text
  def tokenize_text(self, text):
    words=word_tokenize(text)
    return words
  # Remove stopwords
  def remove_stopwords(self, tokens):
    stopwrds=stopwords.words('english')
    token_clean=[token for token in tokens if token not in stopwrds and len(token)>3]
    return token_clean
  # stem the tokens
  def stem_tokens(self, tokens):
   stemmed_tokens=[PorterStemmer().stem(token) for token in tokens]
   return stemmed_tokens

Preprocess the text data

In [ ]:
tp=text_preprocessing()
lower=tr_data.iloc[:,:1].apply(lambda row:tp.convert_to_lower(row.str), axis=1)
sen_cleaned=lower.apply(lambda row:tp.remove_punctuation(row.values), axis=1)
#tokens=sen_cleaned.apply(lambda row:tp.tokenize_text(row))
#cleaned_tokens=tokens.apply(lambda row:tp.remove_stopwords(row))
#stemmed_tokens=cleaned_tokens.apply(lambda row:tp.stem_tokens(row))

In [ ]:
tst_data=pd.read_csv('/content/Test.csv')
tst_data.shape

(5000, 2)

In [ ]:
# preprocess the test data
lower=tst_data.iloc[:,:1].apply(lambda row:tp.convert_to_lower(row.str), axis=1)
sen_cleaned_tst=lower.apply(lambda row:tp.remove_punctuation(row.values), axis=1)


In [ ]:
full_text_df=pd.concat([sen_cleaned,sen_cleaned_tst])
full_text_df.shape

(45000,)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tv=TfidfVectorizer( analyzer='word',ngram_range=(1,1),sublinear_tf=True)

In [ ]:
full_vocab=tv.fit(full_text_df)
tr_X=tv.transform(sen_cleaned)


In [ ]:
y_train=tr_data['label']
y_train.head()

0    0
1    0
2    0
3    0
4    1
Name: label, dtype: int64

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

Linear SVM to classify the sentiment

In [ ]:
SVM=LinearSVC()
SVM.fit(tr_X,y_train.values)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
y_tst=tst_data['label']
test_X=tv.transform(sen_cleaned_tst)


In [ ]:
y_tst_pred=SVM.predict(test_X)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_tst, y_tst_pred)

0.8994